In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import os

# --- [1] 경로 및 파일 설정 ---
DATA_PATH = '../../data/raw/'
OOF_PATH = './oof_data/'
SUB_PATH = './submissions/'

# 모델별 OOF 파일 (Large=Voted 방향)
M1_OOF = 'exp11_sota_AUC_0.77212.npy'
M4_OOF = 'exp15_snn_AUC_0.77330.npy'
M7_OOF = 'exp25_m7_Refined_AUC_0.77375.npy'

# 모델별 Submission 파일 (Small=Voted 방향)
M1_SUB = '0130-1923.csv'
M4_SUB = 'Diversity_Focal_SNN_AUC_0.7708.csv'
M7_SUB = 'm7_Refined_test_preds.csv'

# --- [2] 데이터 로드 및 인덱스 정합성 확보 ---
train_df = pd.read_csv(f'{DATA_PATH}train.csv')
a_cols = [f'Q{i}A' for i in 'abcdefghijklmnopqrst']

# m1, m4 인덱스 (familysize <= 50)
m1_idx = train_df[train_df.familysize <= 50].index

# m7 인덱스 (familysize <= 50 & std > 0)
m7_mask = (train_df.familysize <= 50) & (train_df[a_cols].std(axis=1) > 0)
m7_idx = train_df[m7_mask].index

# 정답셋 (방향: Large=Voted)
train_y = (2 - train_df.loc[m7_idx, 'voted'].to_numpy()).astype(np.float32)

# --- [3] OOF 정렬 및 m1+m4 가중치 블렌딩 ---
# m1, m4 OOF 로드 및 m7 인덱스에 맞춤
m1_oof = np.load(f'{OOF_PATH}{M1_OOF}')
m1_aligned = pd.DataFrame(m1_oof, index=m1_idx, columns=['val']).reindex(m7_idx)['val'].values

m4_oof = np.load(f'{OOF_PATH}{M4_OOF}')
m4_aligned = pd.DataFrame(m4_oof, index=m1_idx, columns=['val']).reindex(m7_idx)['val'].values

# m1 + m4 가중치 블렌딩 (92:8) - 확률값 공간
m1_m4_blend_oof = (0.92 * m1_aligned) + (0.08 * m4_aligned)

# --- [4] m7과 최종 랭크 앙상블 ---
m7_oof = np.load(f'{OOF_PATH}{M7_OOF}')

# 순위 변환
rank_m1m4 = pd.Series(m1_m4_blend_oof).rank(pct=True).values
rank_m7 = pd.Series(m7_oof).rank(pct=True).values

# 5:5 랭크 결합 (가장 일반화 성능이 좋았던 비율)
final_ensemble_oof = (rank_m1m4 + rank_m7) / 2
final_auc = roc_auc_score(train_y, final_ensemble_oof)

print('-' * 50)
print(f'exp31: (m1+m4 92:8 Blend) Rank m7 (5:5)')
print(f'Local AUC: {final_auc:.5f}')
print('-' * 50)

# --- [5] 테스트 데이터 적용 및 제출 파일 생성 ---
def get_test_probs(sub_name):
    # Small=Voted(1, 2)를 Large=Voted(1, 0) 확률 방향으로 변환
    return 2.0 - pd.read_csv(f'{SUB_PATH}{sub_name}')['voted']

# 1. m1, m4 블렌딩
m1_test = get_test_probs(M1_SUB)
m4_test = get_test_probs(M4_SUB)
m1_m4_test_blend = (0.92 * m1_test) + (0.08 * m4_test)

# 2. 랭크 변환
test_rank_m1m4 = m1_m4_test_blend.rank(pct=True).values
test_rank_m7 = get_test_probs(M7_SUB).rank(pct=True).values

# 3. 최종 결합 (5:5)
final_test_rank = (test_rank_m1m4 + test_rank_m7) / 2

# 제출 파일 생성 (방향: Small=Voted 복구)
sub = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sub['voted'] = 2.0 - final_test_rank
output_name = f'{SUB_PATH}exp31_m1m4_rank_m7_{final_auc:.5f}.csv'
sub.to_csv(output_name, index=False)

print(f'Submission file created: {output_name}')

--------------------------------------------------
exp31: (m1+m4 92:8 Blend) Rank m7 (5:5)
Local AUC: 0.77373
--------------------------------------------------
Submission file created: ./submissions/exp31_m1m4_rank_m7_0.77373.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import os

# --- [1] 경로 및 파일 설정 ---
DATA_PATH = '../../data/raw/'
OOF_PATH = './oof_data/'

M1_OOF_FILE = 'exp11_sota_AUC_0.77212.npy'
M4_OOF_FILE = 'exp15_snn_AUC_0.77330.npy'
M7_OOF_FILE = 'exp25_m7_Refined_AUC_0.77375.npy'

# --- [2] 데이터 로드 및 인덱스 정합성 확보 ---
train_df = pd.read_csv(f'{DATA_PATH}train.csv')
a_cols = [f'Q{i}A' for i in 'abcdefghijklmnopqrst']

# 인덱스 정의
m1_idx = train_df[train_df.familysize <= 50].index
m7_mask = (train_df.familysize <= 50) & (train_df[a_cols].std(axis=1) > 0)
m7_idx = train_df[m7_mask].index

# 정답셋 (m7 인덱스 기준)
train_y = (2 - train_df.loc[m7_idx, 'voted'].to_numpy()).astype(np.float32)

# OOF 로드 및 m7 인덱스로 정렬 (Reindex)
m1_oof = np.load(f'{OOF_PATH}{M1_OOF_FILE}')
m1_aligned = pd.DataFrame(m1_oof, index=m1_idx, columns=['val']).reindex(m7_idx)['val'].values

m4_oof = np.load(f'{OOF_PATH}{M4_OOF_FILE}')
m4_aligned = pd.DataFrame(m4_oof, index=m1_idx, columns=['val']).reindex(m7_idx)['val'].values

m7_oof = np.load(f'{OOF_PATH}{M7_OOF_FILE}') # m7은 이미 해당 인덱스임

# --- [3] 랭크 변환 ---
# m1+m4를 먼저 92:8로 블렌딩한 후 랭크화
m1_m4_blend = (0.92 * m1_aligned) + (0.08 * m4_aligned)
r_m1m4 = pd.Series(m1_m4_blend).rank(pct=True).values
r_m7 = pd.Series(m7_oof).rank(pct=True).values

# --- [4] 가중치 스캔 (Weight Scan) ---
print(f"{'Weight (m1+m4)':>15} | {'Weight (m7)':>12} | {'Local AUC':>12}")
print("-" * 45)

best_w = 0
max_auc = 0

# 0.00부터 1.00까지 0.01 간격으로 조사
for w in np.linspace(0, 1, 101):
    # 앙상블 공식: w * (m1+m4_rank) + (1-w) * m7_rank
    ensemble_rank = (w * r_m1m4) + ((1 - w) * r_m7)
    auc = roc_auc_score(train_y, ensemble_rank)
    
    # 0.1 단위마다 로그 출력
    if round(w, 2) % 0.1 == 0:
        print(f"{w:15.2f} | {1-w:12.2f} | {auc:12.5f}")
    
    if auc > max_auc:
        max_auc = auc
        best_w = w

print("-" * 45)
print(f"결과: 최적 가중치 m1+m4 = {best_w:.2f}, m7 = {1-best_w:.2f}")
print(f"최고 Local AUC: {max_auc:.5f}")

 Weight (m1+m4) |  Weight (m7) |    Local AUC
---------------------------------------------
           0.00 |         1.00 |      0.77375
           0.10 |         0.90 |      0.77387
           0.20 |         0.80 |      0.77393
           0.40 |         0.60 |      0.77385
           0.80 |         0.20 |      0.77299
---------------------------------------------
결과: 최적 가중치 m1+m4 = 0.24, m7 = 0.76
최고 Local AUC: 0.77393


In [3]:
import pandas as pd
import numpy as np
import os

# --- [1] 경로 및 파일 설정 ---
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

M1_SUB = '0130-1923.csv'
M4_SUB = 'Diversity_Focal_SNN_AUC_0.7708.csv'
M7_SUB = 'm7_Refined_test_preds.csv'

# --- [2] 테스트 데이터 로드 및 랭크 변환 ---
def get_rank(file_name):
    # Small=Voted(1, 2)를 Large=Voted 확률 방향으로 변환 후 랭크 계산
    probs = 2.0 - pd.read_csv(f'{SUB_PATH}{file_name}')['voted']
    return probs.rank(pct=True).values

# m1 + m4 블렌딩 (기존 92:8 비율 유지) 후 랭크화
m1_test_probs = 2.0 - pd.read_csv(f'{SUB_PATH}{M1_SUB}')['voted']
m4_test_probs = 2.0 - pd.read_csv(f'{SUB_PATH}{M4_SUB}')['voted']
m1_m4_blend_probs = (0.92 * m1_test_probs) + (0.08 * m4_test_probs)
rank_m1m4 = pd.Series(m1_m4_blend_probs).rank(pct=True).values

# m7 랭크화
rank_m7 = get_rank(M7_SUB)

# --- [3] 최적 가중치 적용 (24:76) ---
# Ensemble = 0.24 * Rank(m1+m4) + 0.76 * Rank(m7)
final_rank = (0.24 * rank_m1m4) + (0.76 * rank_m7)

# --- [4] 제출 파일 생성 ---
sub = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sub['voted'] = 2.0 - final_rank # 방향 복구
output_name = f'{SUB_PATH}exp31_v2_opt_weight_0.77393.csv'
sub.to_csv(output_name, index=False)

print(f"최적 가중치 앙상블 완료: {output_name}")

최적 가중치 앙상블 완료: ./submissions/exp31_v2_opt_weight_0.77393.csv
